This is a repo that was built to scrape HTML tables from [EVehicles DB](https://evehiclesdb.com/category/transport/electric-cars/). 

## Install All Necessary Packages

In [ ]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Testing Out How the Selenium Library Works

In [ ]:
#Create the Driver
driver = webdriver.Chrome(ChromeDriverManager().install())

page_url = "https://evehiclesdb.com/category/transport/electric-cars/"
driver.get(page_url)

In [ ]:
ev_categories = driver.find_elements(By.CLASS_NAME, "brand_list_item")

ev_categories[0].get_attribute('href')

In [ ]:
ev_element = driver.find_elements(By.CLASS_NAME, "top_cars_list_item")

ev_element

# Full Code

## Generate a list of URLs to scrape from

In [ ]:
# Create Driver
driver = webdriver.Chrome(ChromeDriverManager().install())

#Go to the EVs DB page
page_url = "https://evehiclesdb.com/category/transport/electric-cars/"
driver.get(page_url)

#Find EV makes
ev_make = driver.find_elements(By.CLASS_NAME, "brand_list_item")

ev_makes = [] #creates an empty list
for make in ev_make:
    make_url = make.get_attribute('href')
    make_name = make.text
    ev_makes.append({'ev make' : make_name, 'url': make_url})

In [ ]:
ev_model = [] #list for all the EV models

for model in ev_makes:
    driver.get(model['url'])
    
    ev_models = driver.find_elements(By.CLASS_NAME, "top_cars_list_item")
    
       
    for models in ev_models:
        #models_url = 'https://evehiclesdb.com/transport/'+ model +'/'
        ev_model.append({'ev make': model['ev make'], 'model': models.text})

In [ ]:
#This will generate the url for each EV
ev_df = pd.DataFrame(ev_model)
ev_df['url string'] = ev_df['model'].str.split(' ').str.join("-").str.lower()

ev_df['url'] = 'https://evehiclesdb.com/transport/'+ev_df['url string']+'/'
#some of the URL pages in this database have buttons and we need to always make sure we are accessing "Specifications"
#driver.find_element(By.XPATH, '//span[text()="Specifications"]').click()



## Function to Scrape an HTML Table

In [ ]:
def table_scrape(table):
    """Parses an html segment started with tag <table> followed 
    by multiple <tr> (table rows) and inner <td> (table data) tags.
    It returns a list of rows with inner columns.
    """
    def row_scrape(tr, col_tag='td'): #td (data) 
        return [td.get_text(strip=True) for td in tr.find_all(col_tag)]
    rows = []
    trs = table.find_all('tr')
    for tr in trs: #for every table row
        rows.append(row_scrape(tr, 'td')) #data row
    return rows

### Testing a single URL with the table_scrape function

In [ ]:
#Testing a single URL with the scraping function created in the cell before
url_test = 'https://evehiclesdb.com/transport/honda-e/'

response_test = requests.get(url_test)
soup_test = bs(response_test.text, 'html.parser')
ev_table_test = soup_test.find('table', dir='ltr')

test_table = pd.DataFrame(table_scrape(ev_table_test))
#test_table[:3]

In [ ]:
test_table 

## Scrape ALL HTML Tables from ALL the URLs 

In [ ]:
ev_list = list(ev_df['url']) #List of all the urls for each EV

dict_of_evs = {} #initialize empty dictionary
count = 0
for i in ev_list: #While count is less than or equal to the total number of EVs in the database
    url = i #this is the individual url string for each EV
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    
    #ev_table = soup.find('table', dir='ltr')
    if soup.find('div', class_='list-group'):
        ev_table = soup.find('div', class_='list-group')

    #comp_ev_table = pd.DataFrame(table_scrape(ev_table))
    
    df_name = f"df_{count}" #name of each dictionary key i.e. df_1, df_2... df_count
    dict_of_evs[df_name] = pd.DataFrame(table_scrape(ev_table))

    count += 1


## Combine the Scraped EV Dictionary

### Testing Cell

In [ ]:
#Testing Cell: Ignore

# a = dict_of_evs['df_10'].T
# a.columns = a.iloc[0]
# a

# a = dict_of_evs
# test_concat = pd.concat([a['df_1'],a['df_2']],axis=1).sort_values('0')
# test_concat

## Start Here

### Create a dataframe from the created dictionary

In [377]:
#Combine all dictionaries by concating the dataframes and transposing
metric_df = pd.concat(dict_of_evs, axis=1).T
#Makes the column names the same things as the first row
metric_df.columns = metric_df.iloc[0]
metric_df = metric_df.reset_index().drop(columns={'level_0'})

#metric_df.head()

In [378]:
#Further cleaning the new dataframe by dropping duplicate rows
metric_df['Bool'] = metric_df['level_1'].astype(bool)
metric_df = metric_df.drop(columns='level_1')

metric_df = metric_df[metric_df['Bool'] != False]
metric_df = metric_df.drop(columns='Bool')

#metric_df.head()

### Cleaning up the dataframe

In [379]:
columns = ['Brand','Model','Body Style','Car Engine'] #list of the columns that contain strings
#Makes everything lower case
for column in columns: 
    metric_df[column] = metric_df[column].str.lower() 

metric_df = metric_df.reset_index().drop(columns='index')
#metric_df.head()
  

In [380]:
rename_dict = {'Motor power':'Motor Power (kW)',
            'Maximum Torque, Nm':'Maximum Torque (Nm)',
            'Battery Energy, kWh':'Battery Energy (kWh)',
            'Power reserve (NEDC/EPA/WLTP), km':'Power Reserve (km)',
            'Level Charging (230/400/DC), hours':'Time to Charge (hr)',
            'Electrical Acceleration, 0-100 km/h (0-62.1 mph) in sec':'Acceleration from 0 to 100 km/h (sec)',
            'Top Speed, km/h':'Top Speed (km/h)'
            }
metric_df = metric_df.rename(columns= rename_dict)
metric_df = metric_df.drop(columns={'Power Reserve (km)','Car Engine'})
metric_df.head()

"(df_0, 0)",Brand,Model,Body Style,Motor Power (kW),Maximum Torque (Nm),Battery Energy (kWh),Time to Charge (hr),Acceleration from 0 to 100 km/h (sec),Top Speed (km/h)
0,aiways,u5,suv,150,310,63.0,10.0 /\...,7.5,150
1,aston martin,rapide e,sedan,610,950,65,-\r\n /...,4.1,250
2,audi,q4 sportback e-tron 35,suv,125,310,55.0,-\r\n /...,9.0,160
3,audi,q4 e-tron 50 quattro,suv,220,460,82.0,-\r\n /...,6.2,180
4,audi,q4 e-tron 45 quattro,suv,195,425,82.0,-\r\n /...,6.9,180


### The following cells are for transforming all numeric objects to float values.

In [381]:
#Cleaning 'Time to Charge' column
metric_df['Time_clean'] = metric_df['Time to Charge (hr)'].str.strip().str.replace('-\r\n','',regex=True)
metric_df['Time_clean'] = metric_df['Time_clean'].str.strip().str.replace('/\r\n','',regex=True).str.replace(',','.',regex=True).dropna()
metric_df['Time_clean'] = metric_df['Time_clean'].str.extract("([-+]?\d*\.\d+|[-+]?\d+)").astype(float) #extracts the numeric values

metric_df['Time to Charge (hr)'] = metric_df['Time_clean']
metric_df = metric_df.drop(columns={'Time_clean'})

metric_df.head()

"(df_0, 0)",Brand,Model,Body Style,Motor Power (kW),Maximum Torque (Nm),Battery Energy (kWh),Time to Charge (hr),Acceleration from 0 to 100 km/h (sec),Top Speed (km/h)
0,aiways,u5,suv,150,310,63.0,10.00,7.5,150
1,aston martin,rapide e,sedan,610,950,65,3.00,4.1,250
2,audi,q4 sportback e-tron 35,suv,125,310,55.0,8.30,9.0,160
3,audi,q4 e-tron 50 quattro,suv,220,460,82.0,8.15,6.2,180
4,audi,q4 e-tron 45 quattro,suv,195,425,82.0,8.15,6.9,180


In [382]:
#Cleaning 'Motor Power' column
metric_df['Motor_clean'] = metric_df['Motor Power (kW)'].str.strip().str.replace('-\r\n','',regex=True).str.replace('/\r\n','',regex=True).str.replace(',','.',regex=True).dropna()
metric_df['Motor_clean'] = metric_df['Motor_clean'].str.extract("([-+]?\d*\.\d+|[-+]?\d+)").astype(float) #extracts the numeric values

metric_df['Motor Power (kW)'] = metric_df['Motor_clean']
metric_df = metric_df.drop(columns='Motor_clean')


In [383]:
#Cleaning 'Maximum Torque' column
metric_df['Torque_clean'] = metric_df['Maximum Torque (Nm)'].str.strip().str.replace('-\r\n','',regex=True)
metric_df['Torque_clean'] = metric_df['Torque_clean'].str.strip().str.replace('/\r\n','',regex=True).str.replace(',','.',regex=True).dropna()
metric_df['Torque_clean'] = metric_df['Torque_clean'].str.extract("([-+]?\d*\.\d+|[-+]?\d+)").astype(float) #extracts the numeric values

metric_df['Maximum Torque (Nm)'] = metric_df['Torque_clean']
metric_df = metric_df.drop(columns={'Torque_clean'})

In [384]:
#Cleaning 'Battery Energy' column
metric_df['Battery_clean'] = metric_df['Battery Energy (kWh)'].str.strip().str.replace('-\r\n','',regex=True)
metric_df['Battery_clean'] = metric_df['Battery_clean'].str.strip().str.replace('/\r\n','',regex=True).str.replace(',','.',regex=True).dropna()
metric_df['Battery_clean'] = metric_df['Battery_clean'].str.extract("([-+]?\d*\.\d+|[-+]?\d+)").astype(float) #extracts the numeric values

metric_df['Battery Energy (kWh)'] = metric_df['Battery_clean']
metric_df = metric_df.drop(columns={'Battery_clean'})

In [385]:
#Cleaning 'Acceleration' column
metric_df['Acceleration_clean'] = metric_df['Acceleration from 0 to 100 km/h (sec)'].str.strip().str.replace('-\r\n','',regex=True)
metric_df['Acceleration_clean'] = metric_df['Acceleration_clean'].str.strip().str.replace('/\r\n','',regex=True).str.replace(',','.',regex=True).dropna()
metric_df['Acceleration_clean'] = metric_df['Acceleration_clean'].str.extract("([-+]?\d*\.\d+|[-+]?\d+)").astype(float) #extracts the numeric values

metric_df['Acceleration from 0 to 100 km/h (sec)'] = metric_df['Acceleration_clean']
metric_df = metric_df.drop(columns={'Acceleration_clean'})

In [386]:
#Cleaning 'Top Speed' column
metric_df['Speed_clean'] = metric_df['Top Speed (km/h)'].str.strip().str.replace('-\r\n','',regex=True)
metric_df['Speed_clean'] = metric_df['Speed_clean'].str.strip().str.replace('/\r\n','',regex=True).str.replace(',','.',regex=True).dropna()
metric_df['Speed_clean'] = metric_df['Speed_clean'].str.extract("([-+]?\d*\.\d+|[-+]?\d+)").astype(float) #extracts the numeric values

metric_df['Top Speed (km/h)'] = metric_df['Speed_clean']
metric_df = metric_df.drop(columns={'Speed_clean'})

In [ ]:
metric_df.dtypes #Checking that all numeric values return as floats

## Converting from Metric to Imperial units (sigh... America)

In [387]:
imperial_df = metric_df.copy()

imperial_df.head()

"(df_0, 0)",Brand,Model,Body Style,Motor Power (kW),Maximum Torque (Nm),Battery Energy (kWh),Time to Charge (hr),Acceleration from 0 to 100 km/h (sec),Top Speed (km/h)
0,aiways,u5,suv,150.0,310.0,63.0,10.00,7.5,150.0
1,aston martin,rapide e,sedan,610.0,950.0,65.0,3.00,4.1,250.0
2,audi,q4 sportback e-tron 35,suv,125.0,310.0,55.0,8.30,9.0,160.0
3,audi,q4 e-tron 50 quattro,suv,220.0,460.0,82.0,8.15,6.2,180.0
4,audi,q4 e-tron 45 quattro,suv,195.0,425.0,82.0,8.15,6.9,180.0


In [388]:
#We'll create a dataframe that puts everything into Imperial Units
imperial_df['Motor Power (hp)'] = (imperial_df['Motor Power (kW)']*1.341).round(2)
imperial_df['Maximum Torque (ft-lb)'] = (imperial_df['Maximum Torque (Nm)']/1.356).round(2)
imperial_df['Battery Energy (MBTU)'] = (imperial_df['Battery Energy (kWh)']*3412/10000).round(2)
imperial_df['Top Speed (mph)'] = (imperial_df['Top Speed (km/h)']/1.609).round(2)

#Removing all the metric columns
metric_columns = {'Motor Power (kW)',
                'Maximum Torque (Nm)',
                'Battery Energy (kWh)',
                'Top Speed (km/h)',
                }
imperial_df = imperial_df.drop(columns=metric_columns)

In [389]:
imperial_df.head()

"(df_0, 0)",Brand,Model,Body Style,Time to Charge (hr),Acceleration from 0 to 100 km/h (sec),Motor Power (hp),Maximum Torque (ft-lb),Battery Energy (MBTU),Top Speed (mph)
0,aiways,u5,suv,10.00,7.5,201.15,228.61,21.50,93.23
1,aston martin,rapide e,sedan,3.00,4.1,818.01,700.59,22.18,155.38
2,audi,q4 sportback e-tron 35,suv,8.30,9.0,167.62,228.61,18.77,99.44
3,audi,q4 e-tron 50 quattro,suv,8.15,6.2,295.02,339.23,27.98,111.87
4,audi,q4 e-tron 45 quattro,suv,8.15,6.9,261.50,313.42,27.98,111.87


## Exporting the clean dataframe as "ev_specs"

In [391]:
#Create a final dataframe for exporting to .csv file 
ev_specs = imperial_df.copy()

ev_specs.to_csv(r'C:\Users\kagom\OneDrive\Pictures\Familiar\Desktop\DS4A Empowerment\0 Project\ev_specs.csv')